In [1]:
import pysm3
import argparse
from pixell import curvedsky as cs, reproject, enmap
import numpy as np
import healpy as hp
import pickle
import pysm3.units as u
import os
import gc
from solenspipe.utility import w_n

--------------------------------------------------------------------------

  Local host:   cpu-q-346
  Local device: mlx5_0
--------------------------------------------------------------------------


In [2]:
args = argparse.Namespace()

args.mlmax=4000
args.nside=2048

In [3]:
ell = np.arange(args.mlmax+1)
lfac = ell * (ell + 1) / (2 * np.pi)

In [4]:
pysm_str = {'d9': ['d9'],
            'd10': ['d10'],
            'd12': ['d12'],
            's4': ['s4'],
            's5': ['s5'],
            's7': ['s7'],
            'low':['d9', 's4'],
            'medium':['d10', 's5'],
            'high': ['d12','s7']
           }

In [5]:
ACT_FREQ = [90, 150]
SO_FREQ = [27,39,93,145,225,280]

## load masks

and compute w-n factors

In [6]:
ACT_masks = {}
SO_masks = {}

# ACT_masks['GAL060'] = enmap.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/masks/ACT_GAL060_wcsACT_car_apo3deg.fits')
# ACT_masks['GAL070'] = enmap.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/masks/ACT_GAL070_wcsACT_car_apo3deg.fits')
# ACT_masks['GAL080'] = enmap.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/masks/ACT_GAL080_wcsACT_car_apo3deg.fits')
SO_masks['GAL070'] = enmap.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/masks/SOLAT_GAL070_wcsSO_car_apo3deg.fits')
SO_masks['GAL080'] = enmap.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/masks/SOLAT_GAL080_wcsSO_car_apo3deg.fits')

In [7]:
args.shape, args.wcs = SO_masks['GAL070'].shape, SO_masks['GAL070'].wcs

In [8]:
masks = {'ACT': ACT_masks, 'SO': SO_masks}

In [9]:
for key in masks.keys():
    if key == 'ACT':
        continue
    for fsky in masks[key].keys():
        print(f'fsky {key} {fsky} {w_n(masks[key][fsky],1):.3f}')

fsky SO GAL070 0.429
fsky SO GAL080 0.495


In [10]:
w2_factors = {}
for key in masks.keys():
    if key == 'ACT':
        continue
    for fsky in masks[key].keys():
        w2_factors[f'{key}_{fsky}'] = w_n(masks[key][fsky],2)

In [11]:
def get_sky_model(args, str_fg):
    
    '''
        args.nside: 2048 
        args.freq: 
    '''
    
    sky_dust = pysm3.Sky(nside=args.nside, preset_strings=list(str_fg))
    map_freqGHz = sky_dust.get_emission(args.freq * u.GHz)
    sky_hp_maps = map_freqGHz.to(u.uK_CMB, equivalencies=u.cmb_equivalencies(args.freq*u.GHz))
    
    return sky_hp_maps

def sky_model_car(mapa, args):
    '''
        args.shape
        args.wcs
        args.mlmax
    '''
    
    car_map = reproject.healpix2map(mapa, args.shape, args.wcs, lmax = args.mlmax, rot="gal,cel")
    
    return car_map


def get_2pt_car(mapa, mask, w2, args):
    
    fg_alm = cs.map2alm(mapa * mask, lmax=args.mlmax)
    cl_2pts = cs.alm2cl(fg_alm) / w2
    
    return cl_2pts

In [12]:
def get_map_name(fgkey, sim_id, fsky=None):
    tag = f'{fgkey}_{sim_id}'
    if fsky is not None:
        tag = f'{tag}_{fsky}'
    return f'map_car_{tag}_IQU.fits'

In [13]:
args.output_dir = '/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/rawmaps_250103_wcsSO/'
output_path = lambda x: os.path.join(args.output_dir, x)

In [14]:
cl_2pts = {}

In [15]:
# exp='ACT'
# fsky = 'GAL070'
# for freq in ACT_FREQ:
#     args.freq = freq
#     for fg_type in pysm_str.keys():
#         key = f'{args.freq}_{fg_type}'
#         print(key)
#         hp_map = get_sky_model(args, pysm_str[fg_type])
#         car_map = sky_model_car(hp_map, args)
#         enmap.write_map(output_path(get_map_name(key, sim_id=1000)), car_map )
#         cl_2pts[f'{exp}_{key}_{fsky}'] = get_2pt_car(car_map, masks[exp][fsky], w2_factors[f'{exp}_{fsky}'], args)
        
#     del hp_map, car_map
#     gc.collect()

In [16]:
exp='SO'
fsky = 'GAL070'
for freq in SO_FREQ:
    args.freq = freq
    for fg_type in pysm_str.keys():
        key = f'{args.freq}_{fg_type}'
        print(key)
        hp_map = get_sky_model(args, pysm_str[fg_type])
        car_map = sky_model_car(hp_map, args)
        enmap.write_map(output_path(get_map_name(key, sim_id=1000)), car_map )
        cl_2pts[f'{exp}_{key}_{fsky}'] = get_2pt_car(car_map, masks[exp][fsky], w2_factors[f'{exp}_{fsky}'], args)
        
    del hp_map, car_map
    gc.collect()

27_d9
27_d10
27_d12
27_s4
27_s5
27_s7
27_low
27_medium
27_high
39_d9
39_d10
39_d12
39_s4
39_s5
39_s7
39_low
39_medium
39_high
93_d9
93_d10
93_d12
93_s4
93_s5
93_s7
93_low
93_medium
93_high
145_d9
145_d10
145_d12
145_s4
145_s5
145_s7
145_low
145_medium
145_high
225_d9
225_d10
225_d12
225_s4
225_s5
225_s7
225_low
225_medium
225_high
280_d9
280_d10
280_d12
280_s4
280_s5
280_s7
280_low
280_medium
280_high


In [17]:
with open(args.output_dir + '/raw2pts_GAL070_SO.pkl', 'wb') as f:
    pickle.dump(cl_2pts, f)

## DustFilaments

In [18]:
df_path = '/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/dust_sims/'

In [19]:
DF_EB0_path = df_path + 'DustFilaments_TQU_45M_400pc_SOLAT_Dust-gnilc-unires-limit50-sigmatheta14_nside2048_baseline_seed0000_AllScaleMap_f353p0.fits'
DF_EB_path = df_path + 'DustFilaments_TQU_45M_400pc_SOLAT_Dust-gnilc-unires-limit50-sigmatheta14_nside2048_ALD_fiducial_seed0000_AllScaleMap_f353p0.fits'

In [20]:
sky_hp_maps = {'DF_EB0': hp.read_map(DF_EB0_path, field=(0,1,2)),
               'DF_EB': hp.read_map(DF_EB_path, field=(0,1,2))}

In [21]:
# extrapolation other freqs with Carlos' code

In [22]:
H_PLANCK = 6.6260755e-34
K_BOLTZ = 1.380658e-23
T_CMB = 2.72548
def thermo2rj(nu):
    x = H_PLANCK*nu*1.e9/(K_BOLTZ*T_CMB)
    return x**2 * np.exp(x) / (np.expm1(x))**2
def sed_dust(nu, beta, Tdust):
    x_353 = H_PLANCK*353e9/(K_BOLTZ*Tdust)
    x_nu = H_PLANCK*nu*1e9/(K_BOLTZ*Tdust)
    sed_fact_353 = (353e9)**(beta+1) / np.expm1(x_353) / thermo2rj(353.0)
    sed_fact_nu  = (nu * 1e9)**(beta+1) / np.expm1(x_nu) / thermo2rj(nu)
    return sed_fact_nu / sed_fact_353

In [23]:
beta_map = hp.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/Planck/COM_CompMap_Dust-GNILC-Model-Spectral-Index_2048_R2.00.fits')
Tdust_map = hp.read_map('/rds/project/dirac_vol5/rds-dirac-dp002/ia404/fgs/Planck/COM_CompMap_Dust-GNILC-Model-Temperature_2048_R2.00.fits')

In [26]:
sed_factor = {}

for freq in SO_FREQ:
    sed_factor[freq] = np.nan_to_num(sed_dust(freq, beta_map, Tdust_map))

/tmp/ipykernel_2897908/1962930133.py:10: RuntimeWarning: overflow encountered in power
  sed_fact_353 = (353e9)**(beta+1) / np.expm1(x_353) / thermo2rj(353.0)
/tmp/ipykernel_2897908/1962930133.py:10: RuntimeWarning: overflow encountered in divide
  sed_fact_353 = (353e9)**(beta+1) / np.expm1(x_353) / thermo2rj(353.0)
/tmp/ipykernel_2897908/1962930133.py:11: RuntimeWarning: overflow encountered in divide
  sed_fact_nu  = (nu * 1e9)**(beta+1) / np.expm1(x_nu) / thermo2rj(nu)
/tmp/ipykernel_2897908/1962930133.py:12: RuntimeWarning: invalid value encountered in divide
  return sed_fact_nu / sed_fact_353
/tmp/ipykernel_2897908/1962930133.py:11: RuntimeWarning: overflow encountered in power
  sed_fact_nu  = (nu * 1e9)**(beta+1) / np.expm1(x_nu) / thermo2rj(nu)


In [27]:
assert np.isfinite(beta_map).all()
assert np.isfinite(Tdust_map).all()
for freq in SO_FREQ:
    print(freq)
    assert np.isfinite(sed_factor[freq]).all()
    

27
39
93
145
225
280


In [28]:
sky_hp_maps_freq = {}

for key in sky_hp_maps.keys():
    for freq in SO_FREQ:
        sky_hp_maps_freq[f'{freq}_{key}'] = sky_hp_maps[key] * sed_factor[freq]

In [29]:
sky_car_maps_freq = {}
for key in sky_hp_maps_freq.keys():
    print(key)
    sky_car_maps_freq[key] = sky_model_car(sky_hp_maps_freq[key], args)

27_DF_EB0
39_DF_EB0
93_DF_EB0
145_DF_EB0
225_DF_EB0
280_DF_EB0
27_DF_EB
39_DF_EB
93_DF_EB
145_DF_EB
225_DF_EB
280_DF_EB


In [30]:
for key in sky_car_maps_freq.keys():
    enmap.write_map(output_path(get_map_name(key, sim_id=1000)), sky_car_maps_freq[key])

In [31]:
# exp = 'ACT'
# fsky = 'GAL070'
# for key in sky_car_maps_freq.keys():
#     cl_2pts[f'{exp}_{freq}_{fsky}'] = get_2pt_car(sky_car_maps_freq[key], masks[exp][fsky], w2_factors[f'{exp}_{fsky}'], args)

In [32]:
exp = 'SO'
fsky = 'GAL070'
for key in sky_car_maps_freq.keys():
    cl_2pts[f'{exp}_{key}_{fsky}'] = get_2pt_car(sky_car_maps_freq[key], masks[exp][fsky], w2_factors[f'{exp}_{fsky}'], args)

In [33]:
with open(args.output_dir + '/raw2pts_GAL070_SO_pysmDF.pkl', 'wb') as f:
    pickle.dump(cl_2pts, f)